In [339]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import NavigableString
import requests
import pandas as pd
import numpy as np
import re
import collections
import time
import os
import sys
import zipfile
from zipfile import ZipFile
import glob
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import random
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]
import seaborn as sns

from pylab import rcParams

import catboost
from catboost import CatBoostRegressor
from catboost import Pool, cv


pd.set_option('display.max_columns', 500)
pd.set_option("max_rows", 20)
#pd.set_option('display.max_colwidth', None)

api_key = '5002b3eb47039688a731795808c619fd'

def csv_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.txt'):
            yield filename

def zip_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.zip'):
            yield filename            


def get_bs_from_html(html):
    return BeautifulSoup(html.text, "html.parser")
source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'


def get_text_from_clean(text, liste, direction= "right"):

    pairs = []
   
    if direction == "right":
        for item in liste:
            try:
                if item in text:
                    pairs.append([item, text.split(item)[1].split()[0]])
                else:
                    pairs.append([item, "none"])
            except:
                pairs.append([item, "none"])
    if direction == "left":
        for item in liste:
            try:
                if item in text:
                    pairs.append([item, text.split(item)[0].split()[-1]])
                else:
                     pairs.append([item, "none"])   
            except:
                pairs.append([item, "none"])
    
    return pairs

def clean_string(liste):
    liste = Flatten(liste)
    liste = " ".join(liste)
    liste = " ".join(liste.split())
    return liste

def get_text_from_html(bs, class_name):
    string_list =[]
    soup = bs.find_all(class_=class_name)
    for entry in soup:
        string_list.append(entry.text)
    return string_list

def Flatten(ul):
    fl = []
    for i in ul:
        if type(i) is list:
            fl += Flatten(i)
        else:
            fl += [i]
    return fl

def get_all_data_from_site(bs, link):
      
    names = ["Wohnung","Zimmergröße","Sonstige","Nebenkosten","Miete","Gesamtmiete","Kaution","Ablösevereinbarung"]
    my_list = get_text_from_html(bs,"col-sm-12 hidden-xs")
    my_list = clean_string(my_list)
    dict1 = dict(get_text_from_clean(my_list,names,"left"))
       
    names = ["frei ab: ", "frei bis: "]
    my_list = get_text_from_html(bs,"col-sm-3")
    my_list = clean_string(my_list)
    dict2 = dict(get_text_from_clean(my_list,names,"right"))
    
    names = [" Zimmer in "]
    my_list = get_text_from_html(bs,"col-sm-6")
    my_list = clean_string(my_list)
    dict3 = dict(get_text_from_clean(my_list,names,"right"))
       
    
    names = ["Malmännliche","weiblich",'height="17"']
    count = []
    for name in names:
        try:
            string = str(bs.find(class_="mr5 detail-list-fav-button display-inline-block hidden-xs create_favourite").next_sibling.next_sibling)
            count.append(string.count(name))
        except:
            count.append("none")
    dict4 = dict(zip(names, count))
      
    
    my_list = get_text_from_html(bs,"ul-detailed-view-datasheet print_text_left")
    my_list = [x.strip() for x in my_list]
    try:
        dict5 = dict(get_text_from_clean(my_list[1],["zwischen"],"left"))
    except:
        dict5 = dict(get_text_from_clean(my_list,["zwischen"],"left"))
        
                                     
    my_list = get_text_from_html(bs,"ul-detailed-view-datasheet print_text_left")
    my_list = [x.strip() for x in my_list]
    try:
        dict8 = dict(get_text_from_clean(my_list[1],["Geschlecht"],"right"))
    except:
        dict8 = dict(get_text_from_clean(my_list,["Geschlecht"],"right"))

    
    item_list = ["glyphicons glyphicons-bath-bathtub noprint",
    "glyphicons glyphicons-wifi-alt noprint",
    "glyphicons glyphicons-car noprint",
    "glyphicons glyphicons-fabric noprint",
    "glyphicons glyphicons-display noprint",
    "glyphicons glyphicons-folder-closed noprint",
    "glyphicons glyphicons-mixed-buildings noprint",
    "glyphicons glyphicons-building noprint",
    "glyphicons glyphicons-bus noprint",
    "glyphicons glyphicons-bed noprint",
    "glyphicons glyphicons-fire noprint"]
    data_list = []
    for item in item_list:
        try:
            data_list.append([item[22:-8], clean_string([bs.find(class_=item).next_sibling.next_sibling.next_sibling])])
        except:
            data_list.append([item[22:-8],"none"])
    dict6 = dict(data_list)
    
    liste= get_text_from_html(bs,"col-sm-4 mb10")
    adress_string = clean_string(liste).replace("Adresse ","").replace("Umzugsfirma beauftragen1","").replace("Umzugsfirma beauftragen 1","")
    dict7 = {"Adresse":adress_string, "Link": link}
    
    names = "Miete pro Tag: "
    my_list = get_text_from_html(bs,"col-sm-5")
    my_list = clean_string(my_list)
    if names in my_list:
        dict9 = {"taeglich":1}
    else:
        dict9 = {"taeglich":0}
    
    div_id = 'popover-energy-certification'
    try:
        cs = clean_string([bs.find(id=div_id).next_sibling])
        dict10 = {"baujahr" : cs}
    except:
        dict10 = {"baujahr" : "none"}
    
    rauchen= "Rauchen nicht erwünscht"
    nichrauchen = "Rauchen überall erlaubt"
    my_list = get_text_from_html(bs,"col-sm-6")
    my_list = clean_string(my_list)
    if rauchen in my_list:
        dict11 = {"rauchen":"raucher"}
    if nichrauchen in my_list:
        dict11 = {"rauchen":"nichtraucher"}
    if rauchen not in my_list and nichrauchen not in my_list:
        dict11 = {"rauchen":"keine_Angabe"}
    
    wg_list = ["Zweck-WG","keine Zweck-WG","Berufstätigen-WG", "gemischte WG","Studenten-WG","Frauen-WG","Azubi-WG"]
    dict12 = []
    for wg in wg_list:
        my_list = get_text_from_html(bs,"col-sm-6")
        my_list = clean_string(my_list)
        if wg in my_list:
            dict12.append([wg,1])
        else:
            dict12.append([wg,0])
    dict12 = dict(dict12)

    dict_list =[dict1,dict2,dict3,dict4,dict5,dict8, dict6,dict7,dict7,dict9,dict10, dict11, dict12]
    for item in dict_list:
        dict1.update(item)
    return dict1

def get_bs_from_html(html):
    return BeautifulSoup(html.text, "html.parser")

def get_bs_from_http(link):
    html = requests.get(link)
    return BeautifulSoup(html.text, "html.parser")

def get_html_request(link):
    return requests.get(link)

def get_html_from_scraper(link):
    payload = {'api_key': api_key, 'url':link}
    html = requests.get('http://api.scraperapi.com', params=payload)
    return html

def get_bs_from_http_scraper(link):
    payload = {'api_key':  api_key, 'url':link}
    html = requests.get('http://api.scraperapi.com', params=payload)
    return BeautifulSoup(html.text, "html.parser")


def get_all_links_from_site(bs):
    try:
        all_links = []
        classes = ["listenansicht1 listenansicht-inactive offer_list_item","listenansicht0 listenansicht-inactive offer_list_item",
                  "listenansicht0 offer_list_item", "listenansicht1 offer_list_item"]
        
        for scrape_class in classes:
            links = bs.findAll(class_=scrape_class)
            for link in links:
                all_links.append(link["adid"][31:])
    except:
        print("something went wrong with get_all_links")
    return all_links

def get_all_links(nr_min_sites = 0, nr_max_sites = 0, sleep_time =0):
    linklist = []
    for i in range(nr_min_sites,nr_max_sites):
        try:
            url = 'https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main.41.0.0.'+ str(i) +'.html'
            linklist.extend(get_all_links_from_site(get_bs_from_http_scraper(url)))
            print(f"{i+1-nr_min_sites} from {nr_max_sites-nr_min_sites} Pages loaded. Thats {(i-nr_min_sites+1)/(nr_max_sites-nr_min_sites):.2%}.\
            Linklist now has {len(linklist)} rows (expexted {(i+1-nr_min_sites)*20})", end='\r')        
            time.sleep(sleep_time)
        except:
            pass
    return linklist   

def merge_dicts(dic1,dic2):
    try:
        dic3 = dict(dic2)
        for k, v in dic1.items():
            dic3[k] = Flatten([dic3[k], v]) if k in dic3 else v
        return dic3
    except:
        return dic1

def csv_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.txt'):
            yield filename

def zip_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.zip'):
            yield filename

def get_bot_and_outdated_links():
    deleted_txt = "Vermutlich ist das Objekt bereits vergeben."
    cap_txt ="dass kein Bot die Website automatisiert aufruft."

    source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    theFiles = list(os.listdir(source_dir))

    file_list_del = []
    file_list_bot =[]
    theDict = dict()
    for i in theFiles: #Calculate size for all files here. 
        theStats = os.stat(source_dir + "\\"+i)
        theDict[i] = theStats
        if theDict[i].st_size < 85000 and theDict[i].st_size > 60000:
            f = source_dir + "\\"+ i
            with open(f, "r") as file:
                a = file.read()
                if (deleted_txt in a):
                    file_list_del.append(i[:-4])
                if (cap_txt in a):
                    file_list_bot.append(i[:-4])

    for item in file_list_del:
        f = source_dir + "\\"+ item + ".txt"
        os.remove(f)

    for item in file_list_bot:
        f = source_dir + "\\"+ item + ".txt"
        os.remove(f)    

    t1 = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv')
    t2 = pd.DataFrame(file_list_del, columns =['Links'])
    t1 = t1.append(t2).drop_duplicates(keep="first")
    t1.to_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv', index=False)

    t1 = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Bot_links.csv')
    t2 = pd.DataFrame(file_list_bot, columns =['Links'])
    t1 = t1.append(t2).drop_duplicates(keep="first")
    t1.to_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Bot_links.csv', index=False)

def zip_my_files():
    source_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    dest_dir = r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files'
    os.chdir(dest_dir)  # To work around zipfile limitations
    for csv_filename in csv_files(source_dir):
        file_root = os.path.splitext(csv_filename)[0]
        zip_file_name = file_root + '.zip'
        zip_file_path = os.path.join(dest_dir, zip_file_name)
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.write(csv_filename)
        os.remove(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Files\\' + csv_filename)
        
def get_housing_links():
    
    get_bot_and_outdated_links()
    zipped_files = []
    for csv_filename in zip_files(source_dir):
        zipped_files.append(csv_filename[:-4])
    zipped_files = pd.DataFrame(zipped_files, columns =['Links'])

    house_links = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\All_Housing_Links.csv')
    outdated_links = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\Outdated_links.csv')
    
    house_links["Links"] = house_links["Links"].str.slice(0,-5)
    house_links = house_links["Links"].append(zipped_files["Links"])
    house_links = house_links.append(outdated_links["Links"])
    
    house_links = house_links.drop_duplicates(keep=False)
    return house_links            

def replace_viertel(x, viertel_liste):
    if x in viertel_liste:
        return x
    elif any([i in x for i in viertel_liste]):
        return [i for (i, v) in zip(viertel_liste, [i in x for i in viertel_liste]) if v][0]
    else:
        return x

def link_to_pandas(full_link):
    stem = full_link[:57]
    link = full_link[57:]
    bs = get_bs_from_http(stem+link)
    data = get_all_data_from_site(bs, link)
    df = pd.DataFrame([data], columns=data.keys())
    df = df.apply(lambda x: x.astype(str).str.lower())
    #rename cols
    df.columns = ['wohnung', 'zimmergröße', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'kaution',
     'ablösevereinbarung', 'frei_ab', 'frei_bis', 'personen', 'männlich', 'weiblich', 'insgesamt', 'geschlecht', 'geschlecht2', 'bath-bathtub', 'wifi-alt', 'car',
     'fabric', 'display', 'folder-closed', 'buildings', 'stock', 'bus', 'bed',
     'fire', 'adresse', 'link', 'taeglich', 'baujahr','rauchen','zweck_wg','keine_zweck_wg','beruf_wg','gemischt_wg','studenten_wg','frauen_wg', 'azubi_wg']
    #remove common words from cols
    remove_list = ["m²","€",r"n.a","none","(",")", ". og","minuten zu fuß entfernt","minute zu fuß entfernt"]
    for col in list(df.columns):
        for item in remove_list:
            df[col] = df[col].str.replace(item,"", regex=False)

    #remove individual words from col
    df["personen"] = df["personen"].str.replace("er","")

    df["straße"]   = df.adresse.str.extract(pat="(.*)\d\d\d\d\d")
    df["straße"]   = df.straße.str.replace("str\.","straße")
    df["straße"]   = df.straße.str.replace("str ","straße")
    df["straße"]   = df.straße.str.replace("strasse","straße")

    # Vermutlich ist möbliert, teilmöbliert = teilmöbliert
    df["bed"]      = df.bed.str.replace("möbliert, teilmöbliert","teilmöbliert")


    df["geschlecht"] = df["geschlecht"] + df["geschlecht2"]
    df["geschlecht"] = df["geschlecht"].str.replace("egalegal","egal")
    # drop unused cols
    #,"Adresse"
    df = df.drop(columns=["geschlecht2"])

    df["folder-closed"] = df["folder-closed"].str.replace("haustiere erlaubt","haustiere")
    df["bath-bathtub"] = df["bath-bathtub"].str.replace("eigenes bad","eigenes_bad")
    df["bath-bathtub"] = df["bath-bathtub"].str.replace("gäste wc","gäste_wc")
    
    
    
    
    class_list =["aufzug, balkon, fahrradkeller, garten, gartenmitbenutzung, haustiere, keller, spülmaschine, terrasse, waschmaschine",
                  "dielen, fliesen, fußbodenheizung, laminat, parkett, pvc, teppich",
                  "badewanne, badmitbenutzung, dusche, eigenes_bad, gäste_wc",
                  "kabel, satellit"]
    
    df = pd.concat([df,df], ignore_index= True)
    
    one_hot_cols = ["folder-closed",'fabric','bath-bathtub',"display"]
    for i, col in enumerate(one_hot_cols):
        df.iloc[1, df.columns.get_loc(col)] = class_list[i]
        df2 = df[col].str.get_dummies(', ')
        df = pd.concat([df, df2.reindex(df.index)], axis=1)
    df = df.head(1)
    
    df = df.set_index("link")
    
    df = df.drop(columns= one_hot_cols)
    df["viertel"] = df.index.to_series().astype(str).str.extract(pat="(.*)\.\d\d\d\d\d\d")
    df["viertel"] = df["viertel"].str.lower()
    repl_viertel = ["--","\d\d\d\d\d","franfurter ","frankfurt am main","franfurt-am-main,""frankfurt-main-","frankfurtnord","frankfurt-","frankfurt","bei-frankfurt","naehe","sudlich-von","u-bahn-station-","-bei-ffm","1-minute-from-","am-main" ]
    for word in repl_viertel:
        df["viertel"] = df["viertel"].str.replace(word,"")
    df["viertel"] = df["viertel"].str.strip('-')

    conditions = [
        (df["frei_ab"] == ""),
        ((df["frei_ab"] != "") & (df["frei_bis"] != "")),
        ((df["frei_ab"] != "") & (df["frei_bis"] == ""))]
    choices = ['inaktiv', 'befristet', 'unbefristet']
    df["status"] = np.select(conditions, choices, default='black')

    df["dauer"] = pd.to_datetime(df.frei_bis, format='%d.%m.%Y', errors='coerce') - pd.to_datetime(df.frei_ab, format='%d.%m.%Y', errors='coerce')
    df['dauer'] = df['dauer'] / np.timedelta64(1, 'D')
    df['dauer'].fillna(0, inplace=True)

    df["wohnung"] = df["wohnung"].str.replace("\.","")
    df["m2_pro_pers"] = pd.to_numeric(df['wohnung'], errors='coerce')/ pd.to_numeric(df['personen'], errors='coerce')

    # Replace uncommon places with common places if they are included in common place
    # Since PLZ is more precise viertel_name will not be used for modelling
    #df["viertel_name"] = df.viertel.apply(replace_viertel, viertel_liste=freq_viertel)
    df_mapped = df
    df_mapped["baujahr"] = df_mapped.baujahr.str.extract(pat="baujahr (\d\d\d\d)")
    df_mapped["PLZ"] = df_mapped.adresse.str.extract("(\d\d\d\d\d)")


    num_cols = ['wohnung', 'zimmergröße', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'bus', 'männlich',
     'personen', 'weiblich', 'kaution', 'ablösevereinbarung', 'personen', 'bus', 'baujahr',"taeglich"]
    for col in num_cols:
        df_mapped[col] = df_mapped[col].astype(str)
        df_mapped[col] = df_mapped[col].str.extract('(\d+)', expand=False)
        df_mapped[col] = df_mapped[col].astype(float)    
    return df    


freq_viertel = ['sachsenhausen', 'bockenheim', 'bornheim', 'nordend-ost', 'nordend-west', 'ostend', 'innenstadt', 'niederrad', 'westend-nord', 'dornbusch', 'gallusviertel', 'gallus', 'bahnhofsviertel', 'westend-sud', 'roedelheim', 'hoechst', 'eschersheim', 'gutleutviertel', 'griesheim', 'oberrad', 'ginnheim', 'heddernheim', 'eckenheim', 'hausen', 'preungesheim', 'flughafen', 'fechenheim', 'altstadt', 'nied', 'bergen-enkheim', 'nieder-eschbach', 'bonames', 'praunheim', 'sossenheim', 'niederursel', 'nordend', 'offenbach', 'seckbach', 'berkersheim', 'kelsterbach', 'unterliederbach', 'sindlingen', 'neu-isenburg', 'schwanheim', 'westend', 'kalbach', 'er-berg', 'europaviertel', 'zeilsheim', 'harheim', 'eschborn', 'riederwald', 'riedberg', 'bad-vilbel', 'goldstein', 'raunheimflughafen', 'nieder-erlenbach', 'oberursel', 'maintal', 'raunheim', 'moerfelden-walldorf', 'kaiserlei', 'nordweststadt', 'langen', 'sachsenhausen-nord', 'eschborn-bei', 'bad-homburg', 'nordend-bornheim', 'rodgau']
plz_list = ['60327', '60326', '65931', '60313', '60325', '60323', '63263', '60322', '63067', '60389', '60486', '65451', '60487', '60311', '60320', '60329', '60318', '65929', '65934', '61440', '63110', '60529', '60488', '60599', '60596', '60314', '61118', '65936', '65719', '60594', '60385', '60388', '65479', '65824', '60598', '60528', '60549', '60386', '65428', '60489', '60438', '60435', '60439', '60433', '60316', '63065', '63069', '60437', '60431', '64546', '65933', '63477', '63225', '61348', '65760']



# Load Data from saved htmls

In [6]:
data_dict ={}
i= 0
for file in zip_files(source_dir):
    i+=1
    print(i, end='\r')
    file_dir = source_dir + "\\" + file
    with ZipFile(file=file_dir) as zip_file:
        bs = zip_file.read(zip_file.namelist()[0]).decode('utf8')
        soup = BeautifulSoup(bs, "html.parser")
        new_dict = get_all_data_from_site(soup,file[:-4])
        if len(data_dict) == 0:
            data_dict = new_dict 
        else:
            data_dict =merge_dicts(data_dict,new_dict)
        if i == 999999:
            break

In [7]:
len(os.listdir(source_dir))

14789

# Convert Dict in DataFrame and transform

In [66]:
df = pd.DataFrame(data_dict,dtype=str)
df = df.apply(lambda x: x.astype(str).str.lower())
#rename cols
df.columns = ['wohnung', 'zimmergröße', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'kaution',
 'ablösevereinbarung', 'frei_ab', 'frei_bis', 'personen', 'männlich', 'weiblich', 'insgesamt', 'geschlecht', 'geschlecht2', 'bath-bathtub', 'wifi-alt', 'car',
 'fabric', 'display', 'folder-closed', 'buildings', 'stock', 'bus', 'bed',
 'fire', 'adresse', 'link', 'taeglich', 'baujahr','rauchen','zweck_wg','keine_zweck_wg','beruf_wg','gemischt_wg','studenten_wg','frauen_wg', 'azubi_wg']

In [67]:
#remove common words from cols
remove_list = ["m²","€",r"n.a","none","(",")", ". og","minuten zu fuß entfernt","minute zu fuß entfernt"]
for col in list(df.columns):
    for item in remove_list:
        df[col] = df[col].str.replace(item,"", regex=False)
        
#remove individual words from col
df["personen"] = df["personen"].str.replace("er","")

df["straße"] = df.adresse.str.extract(pat="(.*)\d\d\d\d\d")
df["straße"] = df.straße.str.replace("str\.","straße")
df["straße"] = df.straße.str.replace("str ","straße")
df["straße"] = df.straße.str.replace("strasse","straße")
df["straße"] = df.straße.str.replace("[^\w\d]","")
df["straße"] = df.straße.str.replace("[0-9]+","")
df["straße"] = df.straße.str.replace("ß","ss")
df.loc[~df["straße"].isin(list(pd.DataFrame(df.straße.value_counts()).query("straße > 10").index)), "straße"] = "no_info_or_rare"

# Vermutlich ist möbliert, teilmöbliert = teilmöbliert
df["bed"]      = df.bed.str.replace("möbliert, teilmöbliert","teilmöbliert")


df["geschlecht"] = df["geschlecht"] + df["geschlecht2"]
df["geschlecht"] = df["geschlecht"].str.replace("egalegal","egal")
# drop unused cols
#,"Adresse"
df = df.drop(columns=["geschlecht2"])

df["folder-closed"] = df["folder-closed"].str.replace("haustiere erlaubt","haustiere")
df["bath-bathtub"] = df["bath-bathtub"].str.replace("eigenes bad","eigenes_bad")
df["bath-bathtub"] = df["bath-bathtub"].str.replace("gäste wc","gäste_wc")


one_hot_cols = ["folder-closed",'fabric','bath-bathtub',"display"]
class_list = [['', 'aufzug','balkon','fahrradkeller', 'garten', 'gartenmitbenutzung','haustiere','keller','spülmaschine','terrasse','waschmaschine'],
              ['', 'dielen','fliesen','fußbodenheizung','laminat','parkett','pvc','teppich'],
              ['', 'badewanne','badmitbenutzung','dusche','eigenes_bad','gäste_wc'],
              ['', 'kabel','satellit']]
for col in one_hot_cols:
    df[col] = df[col].str.replace(",","")
    df[col] = df[col].str.split(" ").str[:]

df_dict = {elem : pd.DataFrame() for elem in one_hot_cols}
mlb = MultiLabelBinarizer()
for i, col in enumerate(df_dict.keys()):
    df_dict[col] = pd.DataFrame(mlb.fit_transform(df[col]),columns=class_list[i], index=df.link)

df = df.set_index("link")
for col in df_dict:
    df = df.join(df_dict[col]).drop("", axis=1)
    
df = df.drop(columns= one_hot_cols)
df["viertel"] = df.index.to_series().astype(str).str.extract(pat="(.*)\.\d\d\d\d\d\d")
df["viertel"] = df["viertel"].str.lower()
repl_viertel = ["--","\d\d\d\d\d","franfurter ","frankfurt am main","franfurt-am-main,""frankfurt-main-","frankfurtnord","frankfurt-","frankfurt","bei-frankfurt","naehe","sudlich-von","u-bahn-station-","-bei-ffm","1-minute-from-","am-main" ]
for word in repl_viertel:
    df["viertel"] = df["viertel"].str.replace(word,"")
df["viertel"] = df["viertel"].str.strip('-')

conditions = [
    (df["frei_ab"] == ""),
    ((df["frei_ab"] != "") & (df["frei_bis"] != "")),
    ((df["frei_ab"] != "") & (df["frei_bis"] == ""))]
choices = ['inaktiv', 'befristet', 'unbefristet']
df["status"] = np.select(conditions, choices)

df["direct_link"]= "https://www.wg-gesucht.de/wg-zimmer-in-Frankfurt-am-Main-" + df.index + ".html"

df["dauer"] = pd.to_datetime(df.frei_bis, format='%d.%m.%Y', errors='coerce') - pd.to_datetime(df.frei_ab, format='%d.%m.%Y', errors='coerce')
df['dauer'] = df['dauer'] / np.timedelta64(1, 'D')
df['dauer'].fillna(0, inplace=True)

df["wohnung"] = df["wohnung"].str.replace("\.","")
df["m2_pro_pers"] = pd.to_numeric(df['wohnung'], errors='coerce')/ pd.to_numeric(df['personen'], errors='coerce')

# Replace uncommon places with common places if they are included in common places
df["viertel_name"] = df.viertel.apply(replace_viertel, viertel_liste=freq_viertel)
df_mapped = df
df_mapped["baujahr"] = df_mapped.baujahr.str.extract(pat="baujahr (\d\d\d\d)")
df_mapped["PLZ"] = df_mapped.adresse.str.extract("(\d\d\d\d\d)")

# Replace uncommon PLZ with new value
# df_mapped.loc[~df_mapped["PLZ"].isin(list(pd.DataFrame(df.PLZ.value_counts()).query("PLZ > 10").index)), "PLZ"] = 99999

num_cols = ['wohnung', 'zimmergröße', 'sonstige', 'nebenkosten', 'miete', 'gesamtmiete', 'bus', 'männlich',
 'personen', 'weiblich', 'kaution', 'ablösevereinbarung', 'personen', 'bus', 'baujahr',"taeglich"]
for col in num_cols:
    df_mapped[col] = df_mapped[col].astype(str)
    df_mapped[col] = df_mapped[col].str.extract('(\d+)', expand=False)
    df_mapped[col] = df_mapped[col].astype(float)



In [42]:
#df_mapped.to_csv (r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\data_df.csv', index = True, header=True)

In [49]:
# Filter outliers
filtered_df = df_mapped.query("taeglich != '1'")
filtered_df = filtered_df.query("gesamtmiete < 1200")
filtered_df = filtered_df.query("gesamtmiete > 150")
filtered_df = filtered_df.query("zimmergröße < 60")
filtered_df = filtered_df.query("zimmergröße > 6")

In [50]:
# Replace na's
#keine angabe
feat_list1 = ["buildings", "stock", "bed", "car", "fire", "straße","PLZ"]
#median
feat_list2 = ["wohnung","bus","personen","m2_pro_pers"]
# null setzen
feat_list3 = ["baujahr","ablösevereinbarung"]

for feat in feat_list1:
    filtered_df[feat].fillna("no_info_or_rare", inplace=True)
    filtered_df[feat] = filtered_df[feat].replace(r'^\s*$', "no_info_or_rare", regex=True)

for feat in feat_list2:
    filtered_df[feat].fillna(filtered_df[feat].median(), inplace=True)
    filtered_df[feat] = filtered_df[feat].replace(r'^\s*$', filtered_df[feat].median(), regex=True)

for feat in feat_list3:
    filtered_df[feat].fillna(0, inplace=True)
    filtered_df[feat] = filtered_df[feat].replace(r'^\s*$', 0, regex=True)


In [284]:
#filtered_df.to_csv (r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\filtered_df.csv', index = True, header=True)
filtered_df = pd.read_csv(r'C:\Users\Thomas.Zoellinger\Documents\Jupyter Notebooks\Housing\filtered_df.csv')
filtered_df.PLZ = filtered_df.PLZ.astype(str) 